The content of this file is heavily inspired by the work done by Shashank Gupta, Ricardo Flores, and Rakesh Bobbala

Please refer to the acknowledgements section of the README file for this project

This file was used to generate the model used to generate predicted Penicillin Concentration in the pipeline to BigQuery Table T_RAMAN_PREDICTION

In [1]:
# load libraries
import numpy as np
import pandas as pd
from chemotools.feature_selection import RangeCut
from chemotools.baseline import LinearCorrection
from chemotools.smooth import SavitzkyGolayFilter
from chemotools.derivative import NorrisWilliams
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
# Want to generate model from data from first 30 batches = first 34325 rows of the dataset
full_data = pd.read_csv("../data/raw/Mendeley_data/100_Batches_IndPenSim_V3.csv", nrows=34325)
full_data.fillna(0, inplace=True)

In [3]:
# Filter to wavelengths
start = 350
end = 1750
rcbw = RangeCut(start, end)
data = rcbw.fit_transform(full_data)
raman_df = pd.DataFrame(data, columns=pd.RangeIndex(start=start, stop=end))

In [36]:
raman_df

,350,351,352,353,354,355,356,357,358,359,...,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34320,106790.0,107040.0,107310.0,107540.0,107740.0,107890.0,108080.0,108370.0,108780.0,109190.0,...,543040.0,543590.0,544200.0,544830.0,545500.0,546270.0,547100.0,547980.0,548890.0,549740.0
34321,92878.0,92968.0,93035.0,93103.0,93111.0,93072.0,93066.0,93200.0,93405.0,93578.0,...,518390.0,518760.0,519190.0,519650.0,520180.0,520770.0,521460.0,522220.0,523030.0,523820.0
34322,105550.0,105580.0,105600.0,105630.0,105660.0,105640.0,105670.0,105850.0,106150.0,106480.0,...,544420.0,544910.0,545460.0,546020.0,546650.0,547380.0,548210.0,549070.0,549960.0,550810.0
34323,99593.0,99842.0,100090.0,100340.0,100540.0,100730.0,100940.0,101230.0,101640.0,102010.0,...,544710.0,545040.0,545380.0,545750.0,546160.0,546650.0,547230.0,547930.0,548660.0,549370.0


In [35]:
full_data.columns[350]

'2089'

In [4]:
# Calculate Derivative
lc = LinearCorrection()
spectra_baseline = lc.fit_transform(raman_df)
sgf = SavitzkyGolayFilter(window_size=15, polynomial_order=2)
spectra_norm = sgf.fit_transform(spectra_baseline)
nw = NorrisWilliams(window_size=15, gap_size=3, derivative_order=1)
spectra_derivative = pd.DataFrame(nw.fit_transform(spectra_norm))

In [5]:
# Create PLS Model
x = spectra_derivative
y = full_data['Penicillin concentration(P:g/L)']

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=1)

In [6]:
# Find best fit model
n_components_list = range(1, 6) 

# Store results
results = []

# Loop over different numbers of latent variables
for n_components in n_components_list:
    # Create and train the PLS model
    pls = PLSRegression(n_components=n_components)
    pls.fit(x_train, y_train)

    # Predict on the test set
    y_pred = pls.predict(x_test)

    # Calculate the root mean squared error
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results.append((n_components, rmse))

# Find the number of components with the lowest error
best_n_components, lowest_rmse = min(results, key=lambda x: x[1])

print("Best number of latent variables:", best_n_components)
print("Lowest RMSE on test set:", lowest_rmse)

# Optionally, you can retrain the model with the best number of components
best_pls_model = PLSRegression(n_components=best_n_components)
best_pls_model.fit(x_train, y_train)

Best number of latent variables: 5
Lowest RMSE on test set: 0.6595161293884361


PLSRegression(n_components=5)

In [16]:
best_pls_model.predict(x_scaled)

array([-0.05909298, -0.05909298, -0.05909298, ..., 26.88867219,
       26.87544667, 25.37794854])

In [18]:
model_prediction = best_pls_model.predict(x_scaled)
pls_results = pd.DataFrame(np.transpose([model_prediction.flatten().tolist(), y]), columns=['predicted', 'actual'])
pls_results.iloc[27321,]

predicted    3.196897
actual       4.014300
Name: 27321, dtype: float64

In [30]:
spectra_norm

array([[    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       ...,
       [ -208.37309132,  -400.51774538,  -634.38164947, ...,
        -1113.58215144,  -737.3827071 ,  -419.57261004],
       [  -48.4852393 ,  -113.61131901,  -190.95262615, ...,
         -771.31886706,  -536.14795701,  -323.89756613],
       [ -138.81242646,  -252.60751021,  -381.84908354, ...,
        -1224.59526035,  -822.84316852,  -477.78847852]])

In [21]:
testp = pls_model.predict(x_scaled)

In [22]:
testpls_results = pd.DataFrame(np.transpose([testp.flatten().tolist(), y]), columns=['predicted', 'actual'])

In [24]:
testpls_results.iloc[27321,]

predicted    3.196897
actual       4.014300
Name: 27321, dtype: float64

In [8]:
joblib.dump(best_pls_model, "../data/processed/model/raman_pls_model.pk1")

['../data/processed/model/raman_pls_model.pk1']